In [40]:
import numpy as np
from tqdm import tqdm_notebook

k = 2

des = str(np.load('prepData/insert2Des__HPGPNRMPC_hg38_chr2.npy'))
anc = str(np.load('prepData/insert2Anc__HPGPNRMPC_hg38_chr2.npy'))

print(len(anc), len(des))

def buildTable(anc, des):
    table = {}
    freq = {}
    for i in tqdm_notebook(range(len(anc)-k*2-1)):
        try:
            table[anc[i:i+k*2+1], des[i+k]] += 1
        except KeyError:
            table[anc[i:i+k*2+1], des[i+k]] = 1
        try:
            freq[anc[i:i+k*2+1]] += 1
        except KeyError:
            freq[anc[i:i+k*2+1]] = 1
    for key in tqdm_notebook(table.keys()):
        table[key] = table[key]/freq[key[0]]
    return table, freq

table, freq = buildTable(anc,des)
    
print(table)
np.save('table_{}'.format(k*2+1), table)

25220017 25220017


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:23: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`



{('AAAAA', 'A'): 0.933412340296086, ('AAAAT', 'A'): 0.9769873607184434, ('AAATT', 'A'): 0.9712559755291513, ('AATTT', 'T'): 0.9651006711409396, ('ATTTT', 'T'): 0.973022004994344, ('TTTTA', 'T'): 0.9740797940797941, ('TTTAA', 'T'): 0.957598798172732, ('TTAAA', 'A'): 0.9513311465405446, ('TAAAG', 'A'): 0.9805083072409805, ('AAAGG', 'A'): 0.9769529170152886, ('AAGGA', 'G'): 0.9601449275362319, ('AGGAA', 'G'): 0.9666151223187026, ('GGAAA', 'A'): 0.9704091075386834, ('GAAAA', 'A'): 0.9733508875471013, ('AAAAC', 'A'): 0.973638066393018, ('AAACT', 'A'): 0.9755783123738172, ('AACTA', 'C'): 0.9612438983336139, ('ACTAT', 'T'): 0.9665436086755884, ('CTATA', 'A'): 0.9320792647903504, ('TATAG', 'C'): 0.050508025327639525, ('ATAGC', 'A'): 0.9764202810018326, ('TAGCC', 'G'): 0.968033495048847, ('AGCCC', 'C'): 0.9566927630322704, ('GCCCT', 'C'): 0.9672685770187095, ('CCCTT', 'A'): 0.005267118133935289, ('CCTTG', 'T'): 0.9749288762446657, ('CTTGT', 'T'): 0.9705575518720296, ('TTGTG', 'G'): 0.954355900

In [41]:
ancName = '_HPGPNRMPC'
def load_seq(chromList):
    inputAll = ''
    predAll = ''
    outputAll = ''
    for chromosome in chromList:
        try:
            inputAll += str(np.load('prepData/insert2Anc_{}_hg38_chr{}.npy'.format(ancName,chromosome)))#[:10000000]
            outputAll += str(np.load('prepData/insert2Des_{}_hg38_chr{}.npy'.format(ancName,chromosome)))#[:10000000]
            predAll += str(np.load('prepData/simulated_{}_-1_chr{}.npy'.format(ancName, chromosome)))#[:10000000]
        except FileNotFoundError:
            print(chromosome)
            continue
        print(len(inputAll), len(outputAll), len(predAll))
        print(inputAll[-10:], outputAll[-10:], predAll[-10:])
    return [inputAll], [outputAll], [predAll]


inputAll, outputAll, predAll = load_seq([12,16,17,19,20,21,22])


22587896 22587896 22587896
GGCACTGTCT AGCACTGTCT GACACTCTCT
42343685 42343685 42343685
GTGGTGGTGT GTGGTGGTAT GTGGTGGTGA
61140191 61140191 61140191
CAAGAACACA CAAGAACACA CAAGAACACA
79887837 79887837 79887837
AAGCATTTGA AAGCATTTGA AAGCATTTGT
104272401 104272401 104272401
AAGATTTTGG AAGATATTGG AAGATTTTGA
129025047 129025047 129025047
TGTGGAAACA TGTGGAAACA TGTGGAAACT
149860432 149860432 149860432
TAGAGGCCAT TGGAGGCCAT TAGAGGCCAC


In [42]:
k = 2
def model_simulate(alphabet, k, inputAll):
    inputAll = inputAll[0]
    table = np.load('table_{}.npy'.format(k*2+1),allow_pickle=True).item()
    pred_table = ''
    for i in tqdm_notebook(range(len(inputAll))):
        if i <k:
            pred_table += inputAll[i]
        elif k<=i<len(inputAll)-k:
            mut_prob = []
            for item in alphabet:
                try:
                    mut_prob.append(table[inputAll[i-k:i+k+1], item])
                except KeyError:
                    mut_prob.append(0)
#             print(mut_prob)
#             print(np.random.choice(len(mut_prob), 1, p=mut_prob))
            pred_table += alphabet[np.random.multinomial(1, mut_prob).argmax()]
#             pred_table += alphabet[np.random.choice(len(mut_prob), 1, p=mut_prob)[0]] 
        else :
            pred_table += inputAll[i]
    return pred_table

In [43]:
mut_dict = np.load('mut_dict_insert2.npy',allow_pickle=True).item()
inv_dict = {v: k for k, v in mut_dict.items()}
print(list(inv_dict.keys()))
alphabet = list(inv_dict.keys())

pred_table = model_simulate(alphabet, k, inputAll)
np.save('predTable_{}'.format(k*2+1), pred_table)
# print(pred_table[:1000])


['A', 'C', 'G', 'T', '-', '0', '^', 'f', '4', 'B', 'w', 't', 'i', 'e', '.', 'c', '>', 'X', '5', '|', '[', 's', 'u', '(', '=', 'q', 'W', '_', '!', '{', '1', 'Q', 'U', 'b', 'M', ';', 'm', '~', 'a', ']', '&', ')', '7', '6', 'p', '%', 'n', 'P', 'F', 'h', '3', ':', 'z', '8', 'R', 'S', '/', 'j', 'o', 'd', '@', 'L', '}', 'H', 'Y', 'x', '"', 'I', '\\', 'J', '9', '`', 'l', '<', 'N', 'V', 'D', '+', 'E', 'Z', '$', '?', '2', 'K', '*', 'g']


/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


In [44]:
np.save('predTable_{}'.format(k*2+1), pred_table)
import math
def cross_entropy(alphabet, k, inputAll, outputAll):
    inputAll = inputAll[0]
    outputAll = outputAll[0]
    table = np.load('table_{}.npy'.format(k*2+1),allow_pickle=True).item()
    result = 0
    count = 0
    for i in tqdm_notebook(range(len(inputAll))):
        if k<=i<len(inputAll)-k:
            try:
                result += -math.log(table[inputAll[i-k:i+k+1], outputAll[i]])
                count +=1
            except KeyError:
#                 print('keyError')

    return result/count

cross_entropy(alphabet, k, inputAll, outputAll)

/home/mcb/users/dlim63/miniconda3/envs/research/lib/python3.7/site-packages/ipykernel_launcher.py:9: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if __name__ == '__main__':


keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

...

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
keyError
k

0.20882172435559532